In [1]:
from nltk.stem import PorterStemmer
ps = PorterStemmer()
import pandas as pd
from nltk.stem import PorterStemmer
import nltk
import contractions
import re

stop_words = nltk.corpus.stopwords.words('english')

In [2]:
import pandas as pd
import numpy as np

pd.options.display.max_colwidth = 200

corpus = ['The sky is blue and beautiful.',
          'Love this blue and beautiful sky!',
          'The quick brown fox jumps over the lazy dog.',
          "A king's breakfast has sausages, ham, bacon, eggs, toast and beans",
          'I love green eggs, ham, sausages and bacon!',
          'The brown fox is quick and the blue dog is lazy!',
          'The sky is very blue and the sky is very beautiful today',
          'The dog is lazy but the brown fox is quick!'    
]
labels = ['weather', 'weather', 'animals', 'food', 'food', 'animals', 'weather', 'animals']

corpus = np.array(corpus)
corpus_df = pd.DataFrame({'Document': corpus, 
                          'Category': labels})
corpus_df = corpus_df[['Document', 'Category']]
corpus_df

,Document,Category
0,The sky is blue and beautiful.,weather
1,Love this blue and beautiful sky!,weather
2,The quick brown fox jumps over the lazy dog.,animals
3,"A king's breakfast has sausages, ham, bacon, eggs, toast and beans",food
4,"I love green eggs, ham, sausages and bacon!",food
5,The brown fox is quick and the blue dog is lazy!,animals
6,The sky is very blue and the sky is very beautiful today,weather
7,The dog is lazy but the brown fox is quick!,animals


In [3]:
def convert_lower(text):
    text = text.lower()
    return text 

def expand_contractions(text):
    text = contractions.fix(text)
    return text 

def remove_special_character(text):
    text = text.replace('...',' ')
    text = re.sub('[^a-zA-Z\s]','',text)
    return text

def stemming(text):
    text = [ps.stem(w) for w in text.split(' ')]
    return ' '.join(text)

def stopword_removal(text):
    text = [w for w in text.split(' ') if w not in stop_words]
    return ' '.join(text)

def special_mention(text):
    text = re.sub('@\w+','',text)
    return text 

In [4]:
def clean_text(text):
    text = convert_lower(text)

    text = expand_contractions(text)

   # text = special_mention(text)

    text = remove_special_character(text)

    text = stemming(text)

    text = stopword_removal(text)
    
    text = text.strip()

    return text

In [5]:
corpus_df['clean_text'] = corpus_df['Document'].apply(lambda x: clean_text(x))

In [6]:
corpus_df

,Document,Category,clean_text
0,The sky is blue and beautiful.,weather,sky blue beauti
1,Love this blue and beautiful sky!,weather,love thi blue beauti sky
2,The quick brown fox jumps over the lazy dog.,animals,quick brown fox jump lazi dog
3,"A king's breakfast has sausages, ham, bacon, eggs, toast and beans",food,king breakfast ha sausag ham bacon egg toast bean
4,"I love green eggs, ham, sausages and bacon!",food,love green egg ham sausag bacon
5,The brown fox is quick and the blue dog is lazy!,animals,brown fox quick blue dog lazi
6,The sky is very blue and the sky is very beautiful today,weather,sky veri blue sky veri beauti today
7,The dog is lazy but the brown fox is quick!,animals,dog lazi brown fox quick


In [7]:
doc = list(corpus_df['clean_text'].values)
doc

['sky blue beauti',
 'love thi blue beauti sky',
 'quick brown fox jump lazi dog',
 'king breakfast ha sausag ham bacon egg toast bean',
 'love green egg ham sausag bacon',
 'brown fox quick blue dog lazi',
 'sky veri blue sky veri beauti today',
 'dog lazi brown fox quick']

In [8]:
corpus = ''
for s in doc:
    corpus = corpus + s

In [9]:
corpus

'sky blue beautilove thi blue beauti skyquick brown fox jump lazi dogking breakfast ha sausag ham bacon egg toast beanlove green egg ham sausag baconbrown fox quick blue dog lazisky veri blue sky veri beauti todaydog lazi brown fox quick'

## Count Vectorizer

In [11]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [14]:
cnt = CountVectorizer()

In [15]:
data = cnt.fit_transform(doc)

In [16]:
data

<8x23 sparse matrix of type '<class 'numpy.int64'>'
	with 45 stored elements in Compressed Sparse Row format>

In [17]:
count_df = pd.DataFrame(data.toarray(),columns=sorted(cnt.vocabulary_.keys()))

In [18]:
count_df.shape

(8, 23)

In [19]:
doc

['sky blue beauti',
 'love thi blue beauti sky',
 'quick brown fox jump lazi dog',
 'king breakfast ha sausag ham bacon egg toast bean',
 'love green egg ham sausag bacon',
 'brown fox quick blue dog lazi',
 'sky veri blue sky veri beauti today',
 'dog lazi brown fox quick']

In [20]:
count_df

,bacon,bean,beauti,blue,breakfast,brown,dog,egg,fox,green,...,king,lazi,love,quick,sausag,sky,thi,toast,today,veri
0,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,1,1,0,0,0,0,0,0,...,0,0,1,0,0,1,1,0,0,0
2,0,0,0,0,0,1,1,0,1,0,...,0,1,0,1,0,0,0,0,0,0
3,1,1,0,0,1,0,0,1,0,0,...,1,0,0,0,1,0,0,1,0,0
4,1,0,0,0,0,0,0,1,0,1,...,0,0,1,0,1,0,0,0,0,0
5,0,0,0,1,0,1,1,0,1,0,...,0,1,0,1,0,0,0,0,0,0
6,0,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,2,0,0,1,2
7,0,0,0,0,0,1,1,0,1,0,...,0,1,0,1,0,0,0,0,0,0


In [21]:
cnt.vocabulary_.keys()

dict_keys(['sky', 'blue', 'beauti', 'love', 'thi', 'quick', 'brown', 'fox', 'jump', 'lazi', 'dog', 'king', 'breakfast', 'ha', 'sausag', 'ham', 'bacon', 'egg', 'toast', 'bean', 'green', 'veri', 'today'])

## Topic Modelling

In [22]:
import gensim

In [23]:
from gensim import corpora

In [25]:
doc_ = [d.split(' ') for d in doc]

In [26]:
dic = corpora.Dictionary(doc_)

In [32]:
dic

In [30]:
doc_term_matrix = [dic.doc2bow(d) for d in doc_]

In [31]:
doc_term_matrix

[[(0, 1), (1, 1), (2, 1)],
 [(0, 1), (1, 1), (2, 1), (3, 1), (4, 1)],
 [(5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1)],
 [(11, 1),
  (12, 1),
  (13, 1),
  (14, 1),
  (15, 1),
  (16, 1),
  (17, 1),
  (18, 1),
  (19, 1)],
 [(3, 1), (11, 1), (14, 1), (16, 1), (18, 1), (20, 1)],
 [(1, 1), (5, 1), (6, 1), (7, 1), (9, 1), (10, 1)],
 [(0, 1), (1, 1), (2, 2), (21, 1), (22, 2)],
 [(5, 1), (6, 1), (7, 1), (9, 1), (10, 1)]]

In [33]:
doc_

[['sky', 'blue', 'beauti'],
 ['love', 'thi', 'blue', 'beauti', 'sky'],
 ['quick', 'brown', 'fox', 'jump', 'lazi', 'dog'],
 ['king', 'breakfast', 'ha', 'sausag', 'ham', 'bacon', 'egg', 'toast', 'bean'],
 ['love', 'green', 'egg', 'ham', 'sausag', 'bacon'],
 ['brown', 'fox', 'quick', 'blue', 'dog', 'lazi'],
 ['sky', 'veri', 'blue', 'sky', 'veri', 'beauti', 'today'],
 ['dog', 'lazi', 'brown', 'fox', 'quick']]

In [34]:
from gensim.models.ldamodel import LdaModel

In [36]:
lda = LdaModel

In [44]:
termsMatrix = lda(doc_term_matrix,num_topics=3, id2word=dic,passes=20)

In [45]:
termsMatrix.print_topics(num_words=5)

[(0,
  '0.098*"love" + 0.098*"ham" + 0.098*"bacon" + 0.098*"sausag" + 0.098*"egg"'),
 (1,
  '0.105*"brown" + 0.105*"quick" + 0.105*"fox" + 0.105*"lazi" + 0.105*"dog"'),
 (2,
  '0.096*"beauti" + 0.095*"sky" + 0.095*"blue" + 0.054*"bean" + 0.054*"breakfast"')]

In [40]:
termsMatrix.print_topics(num_words=3)

[(0, '0.084*"sky" + 0.078*"blue" + 0.067*"beauti"'),
 (1, '0.086*"lazi" + 0.086*"quick" + 0.077*"brown"')]

### POS Tagging - Useful / Implied in Language Translation Models 

In [46]:
import spacy

In [47]:
nlp = spacy.load('en_core_web_sm')

In [48]:
s = 'Google is used by language translations and is an application of NLP .'

In [50]:
s = 'Sundar Pichai is the CEO of Google. '

In [51]:
for t in nlp(s):
    print(t.text, '---> ',  t.pos_, ' ----> ', t.tag_)

Sundar --->  PROPN  ---->  NNP
Pichai --->  PROPN  ---->  NNP
is --->  AUX  ---->  VBZ
the --->  DET  ---->  DT
CEO --->  NOUN  ---->  NN
of --->  ADP  ---->  IN
Google --->  PROPN  ---->  NNP
. --->  PUNCT  ---->  .


## NER Named Entity Recognition 

In [52]:
import nltk

In [53]:
from nltk.tag.stanford import StanfordNERTagger

In [56]:
!wget http://nlp.stanford.edu/software/stanford-corenlp-latest.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [57]:
model = './stanford-corenlp-4.5.4-models-english/edu/stanford/nlp/models/ner/english.all.3class.caseless.distsim.crf.ser.gz'
jar='./stanford-corenlp-4.5.4-models-english.jar'

In [63]:
tagger = StanfordNERTagger('english.all.3class.distsim.crf.ser.gz')

LookupError: 

===========================================================================
  NLTK was unable to find stanford-ner.jar! Set the CLASSPATH
  environment variable.

  For more information, on stanford-ner.jar, see:
    <https://nlp.stanford.edu/software>
===========================================================================